In [1]:
import napari
import napari_workflows as nw
from napari_workflows import Workflow
from skimage.io import imread
import inspect
import magicgui
from napari_workflows._napari_loading import *

file_name = r'C:/Users/ryans/Desktop/blobs.tif'
blobs = imread(file_name)

In [2]:
from napari_workflows._io_yaml_v1 import load_workflow, save_workflow        
filename = 'workflow from napari.yaml'

workflow = load_workflow(filename)
print(workflow)

Workflow:
Result of Connected component labeling (scikit-image, nsbatwm) <- (<function connected_component_labeling at 0x0000022612951F30>, 'Result of Expand labels (scikit-image, nsbatwm)', False)
Result of Expand labels (scikit-image, nsbatwm) <- (<function expand_labels at 0x0000022612952200>, 'Result of Threshold (Li et al 1993, scikit-image, nsbatwm)', 4.0)
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x00000226129512D0>, 'blobs', 4.0)
Result of Percentile (scipy, nsbatwm) <- (<function percentile_filter at 0x0000022612951630>, 'Result of Gaussian (scikit-image, nsbatwm)', 2.0, 4.0)
Result of Threshold (Li et al 1993, scikit-image, nsbatwm) <- (<function threshold_li at 0x0000022612950DC0>, 'Result of Percentile (scipy, nsbatwm)')



In [3]:
old_to_new_name_mapping = old_wf_names_to_new_mapping(workflow)
old_to_new_name_mapping

{'Result of Connected component labeling (scikit-image, nsbatwm)': 'connected_component_labeling result',
 'Result of Expand labels (scikit-image, nsbatwm)': 'expand_labels result',
 'Result of Gaussian (scikit-image, nsbatwm)': 'gaussian_blur result',
 'Result of Percentile (scipy, nsbatwm)': 'percentile_filter result',
 'Result of Threshold (Li et al 1993, scikit-image, nsbatwm)': 'threshold_li result'}

In [6]:
def initialise_root_functions(workflow, viewer):
    root_functions = wf_steps_with_root_as_input(workflow)
    
    for i, wf_step_name in enumerate(root_functions):
        func = workflow._tasks[wf_step_name][0]
        args = workflow._tasks[wf_step_name][1:] 

        signat = signature_w_kwargs_from_function(func, args)
        func.__signature__ = signat

        widget = make_flexible_gui(func, viewer, autocall = True)
        viewer3.window.add_dock_widget(widget)

viewer3 = napari.Viewer()
viewer3.add_image(blobs)
initialise_root_functions(workflow,viewer3)

C:\Users\ryans\anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [5]:
def load_remaining_workflow(workflow, viewer, name_mapping, root_functions):
    root_functions = wf_steps_with_root_as_input(workflow)
    layers = viewer.layers
    
    for root in root_functions:
        followers = workflow.followers_of(root)

        for follower in followers:
            layer_names = [str(lay) for lay in layers]
            sources = workflow.sources_of(follower)
            for source in sources:
                if old_to_new_name_mapping[source] not in layer_names:
                    root_functions.append(root)
                    break
                else:
                    source = sources[0]


                    func = workflow._tasks[follower][0]
                    args = workflow._tasks[follower][1:]

                    signat = signature_w_kwargs_from_function(func, args)
                    func.__signature__ = signat

                    widget = make_flexible_gui(func, viewer3)
                    viewer.window.add_dock_widget(widget, name = func.__name__)

                    set_choices(workflow= workflow,
                                wf_step= follower,
                                viewer= viewer,
                                widget= widget,
                                old_wf_names_to_new_mapping= name_mapping)

                    widget(layers[name_mapping[source]].data)

                    new_follower = workflow.followers_of(follower)
                    followers += new_follower

load_remaining_workflow(workflow, viewer3)